# Week 3: Keyword Search First - The Critical Foundation

> ** The 90% Problem:** Most RAG systems jump straight to vector search and miss the foundation that powers the best retrieval systems. We're doing it right!

## ESSENTIAL SETUP - Do This First!

**Before running any cells, ensure your environment is properly configured:**

```bash
# 1. CRITICAL: Copy the environment configuration
cp .env.example .env

# 2. Verify these Week 3 settings are in your .env:
# OPENSEARCH__HOST=http://opensearch:9200
# OPENSEARCH__INDEX_NAME=arxiv-papers
# ARXIV__MAX_RESULTS=15
```

**Important:** Week 3 requires the `.env` file for OpenSearch connectivity and service configuration. The defaults in `.env.example` work perfectly out of the box!

**Why Keyword Search First?**
- **Exact Match Power:** Find specific technical terms and paper IDs precisely
- **Speed & Efficiency:** BM25 is fast and doesn't require expensive embedding models
- **Interpretable:** You understand exactly why papers were retrieved
- **Production Reality:** Companies like Elasticsearch use keyword search as their foundation

---

# Week 3: OpenSearch Integration & BM25 Search

**What We're Building This Week:**

Week 3 focuses on implementing OpenSearch integration for full-text search capabilities using BM25 scoring. This transforms our system from a simple storage solution into a searchable knowledge base.

## Week 3 Focus Areas

### Core Objectives
- **OpenSearch Integration**: Connect our FastAPI application to OpenSearch cluster
- **Index Management**: Create and manage the arxiv-papers index with proper mappings
- **BM25 Search**: Implement full-text search with relevance scoring
- **Data Pipeline**: Transfer papers from PostgreSQL to OpenSearch
- **Search API**: Expose search functionality through REST endpoints

### What We'll Test In This Notebook
1. **Infrastructure Verification** - Ensure all services from Week 1-2 are running
2. **OpenSearch Service Integration** - Test client creation and health checks
3. **Index Creation & Management** - Create arxiv-papers index with proper mappings
4. **Data Pipeline** - Transfer papers from PostgreSQL to OpenSearch
5. **BM25 Search Functionality** - Test search queries with relevance scoring
6. **Search API Endpoints** - Verify FastAPI search endpoints work correctly

### Success Metrics
- OpenSearch cluster healthy and accessible
- arxiv-papers index created with proper mappings
- Papers successfully indexed from PostgreSQL
- BM25 search returns relevant results with scores
- Search API endpoints respond correctly
- All components ready for production use

---

## Week 3 Component Status
| Component | Purpose | Status |
|-----------|---------|--------|
| **OpenSearch Client** | Connect to OpenSearch cluster | ✅ Complete |
| **Index Management** | Create and manage search indices | ✅ Complete |
| **Query Builder** | Build complex search queries | ✅ Complete |
| **Data Pipeline** | Transfer papers to OpenSearch | ✅ Complete |
| **Search API** | REST endpoints for search | ✅ Complete |
| **BM25 Scoring** | Relevance-based search results | ✅ Complete |

## IMPORTANT: Week 3 Docker Services Restart

**NEW USERS OR INTEGRATION CONFLICTS**: Week 3 introduces OpenSearch integration that requires fresh container state. Use this clean restart approach:

### Fresh Start (Recommended for Week 3)
```bash
# Complete clean slate - removes all data but ensures correct OpenSearch state
docker compose down -v

# Build fresh containers with latest code
docker compose up --build -d
```

**When to use this:**
- First time running Week 3 
- OpenSearch connection issues
- Index conflicts or mapping errors
- Want to start with clean OpenSearch state

**Note**: This destroys existing data but ensures you have the correct Week 3 configuration with proper OpenSearch integration.

---

## Prerequisites Check

**Before starting:**
1. Week 1 infrastructure completed
2. Week 2 arXiv integration working
3. UV environment activated
4. Docker Desktop running
5. Some papers already in PostgreSQL from Week 2

**Why fresh containers?** Week 3 includes OpenSearch integration that requires proper cluster initialization and may conflict with existing index states.

**Service Access Points:**
- **FastAPI**: http://localhost:8000/docs (API documentation)
- **PostgreSQL**: via API or `docker exec -it rag-postgres psql -U rag_user -d rag_db`
- **OpenSearch**: http://localhost:9200/_cluster/health
- **Ollama**: http://localhost:11434 (LLM service)
- **Airflow**: http://localhost:8080 (Username: `admin`, Password: `admin`)

## Environment Setup

In [1]:
# Environment Setup and Path Configuration
import sys
from pathlib import Path
import json
import requests # type: ignore

print(f"Python Version: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
print(f"Environment: {sys.executable}")

# Find project root and add to Python path
current_dir = Path.cwd()
if current_dir.name == "week3" and current_dir.parent.name == "notebooks":
    project_root = current_dir.parent.parent
elif (current_dir / "compose.yml").exists():
    project_root = current_dir
else:
    project_root = None

if project_root and (project_root / "compose.yml").exists():
    print(f"Project root: {project_root}")
    sys.path.insert(0, str(project_root))
else:
    print("Missing compose.yml - check directory")
    exit()

Python Version: 3.12.3
Environment: /home/bhargav/arxiv-paper-curator/.venv/bin/python3
Project root: /home/bhargav/arxiv-paper-curator


## 1. Infrastructure Verification

In [2]:
# Service Health Verification
print("WEEK 3 PREREQUISITE CHECK")
print("=" * 50)

services_to_test = {
    "FastAPI": "http://localhost:8000/api/v1/health",
    "PostgreSQL (via API)": "http://localhost:8000/api/v1/health", 
    "OpenSearch": "http://localhost:9200/_cluster/health",
    "Airflow": "http://localhost:8080/health"  
}

all_healthy = True

for service_name, url in services_to_test.items():
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print(f"✓ {service_name}: Healthy")
        else:
            print(f"✗ {service_name}: HTTP {response.status_code}")
            all_healthy = False
    except requests.exceptions.ConnectionError:
        print(f"✗ {service_name}: Not accessible")
        all_healthy = False
    except Exception as e:
        print(f"✗ {service_name}: {type(e).__name__}")
        all_healthy = False

print()
if all_healthy:
    print("All services healthy! Ready for Week 3 OpenSearch integration.")
else:
    print("Some services need attention. Please run: docker compose up --build")

WEEK 3 PREREQUISITE CHECK
✓ FastAPI: Healthy
✓ PostgreSQL (via API): Healthy
✓ OpenSearch: Healthy
✓ Airflow: Healthy

All services healthy! Ready for Week 3 OpenSearch integration.


## 2. OpenSearch Client Setup

In [3]:
# OpenSearch Client Setup
from src.services.opensearch.factory import make_opensearch_client
from opensearchpy import OpenSearch

print("OPENSEARCH CLIENT SETUP")
print("=" * 40)

# Create OpenSearch client using factory pattern
opensearch_client = make_opensearch_client()

# Override for notebook execution (localhost instead of container hostname)
opensearch_client.host = "http://localhost:9200"
opensearch_client.client = OpenSearch(
    hosts=["http://localhost:9200"],
    http_compress=True,
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
)

print(f"Client configured with host: {opensearch_client.host}")
print(f"Index name: {opensearch_client.index_name}")

# Test health check
is_healthy = opensearch_client.health_check()
if is_healthy:
    print("✓ OpenSearch health check: PASSED")
else:
    print("✗ OpenSearch health check: FAILED")

OPENSEARCH CLIENT SETUP
Client configured with host: http://localhost:9200
Index name: arxiv-papers-chunks
✓ OpenSearch health check: PASSED


## Index Configuration

In [4]:
# Display Index Configuration
# Display Index Configuration
from src.services.opensearch.index_config_hybrid import (
    ARXIV_PAPERS_CHUNKS_INDEX,
    ARXIV_PAPERS_CHUNKS_MAPPING,
    HYBRID_RRF_PIPELINE
)

print("INDEX CONFIGURATION")
print("=" * 40)
print(f"Index Name: {ARXIV_PAPERS_CHUNKS_INDEX}")

print(f"\nKey Features:")
print("• Custom text analyzers for better search")
print("• BM25 scoring for relevance")
print("• Vector embeddings for semantic search (1024 dimensions)")
print("• Hybrid search with RRF (Reciprocal Rank Fusion)")
print("• HNSW algorithm for fast vector search")

print("\nMapping Structure:")
print(json.dumps(ARXIV_PAPERS_CHUNKS_MAPPING, indent=2))

print("\nHybrid RRF Pipeline:")
print(json.dumps(HYBRID_RRF_PIPELINE, indent=2))

INDEX CONFIGURATION
Index Name: arxiv-papers-chunks

Key Features:
• Custom text analyzers for better search
• BM25 scoring for relevance
• Vector embeddings for semantic search (1024 dimensions)
• Hybrid search with RRF (Reciprocal Rank Fusion)
• HNSW algorithm for fast vector search

Mapping Structure:
{
  "settings": {
    "number_of_shards": 1,
    "number_of_replicas": 0,
    "index.knn": true,
    "index.knn.space_type": "cosinesimil",
    "analysis": {
      "analyzer": {
        "standard_analyzer": {
          "type": "standard",
          "stopwords": "_english_"
        },
        "text_analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "stop",
            "snowball"
          ]
        }
      }
    }
  },
  "mappings": {
    "dynamic": "strict",
    "properties": {
      "chunk_id": {
        "type": "keyword"
      },
      "arxiv_id": {
        "type": "keyword"
      },
      "paper_id":

### Create Index

In [5]:
# Create Index if it doesn't exist
print("INDEX CREATION")
print("=" * 40)

try:
    # Check if index already exists
    index_exists = opensearch_client.client.indices.exists(index=opensearch_client.index_name)
    
    if index_exists:
        print(f"✓ Index '{opensearch_client.index_name}' already exists")
        
        # Get current index statistics
        stats = opensearch_client.get_index_stats()
        if stats and 'error' not in stats:
            print(f"\nCurrent Statistics:")
            print(f"   Documents: {stats.get('document_count', 0)}")
            print(f"   Size: {stats.get('size_in_bytes', 0):,} bytes")
    else:
        print(f"Creating new index: {opensearch_client.index_name}")
        
        # Create the index with our custom mapping
        success = opensearch_client.create_index()
        
        if success:
            print(f"✓ Index created successfully!")
        else:
            print(f"✗ Index creation failed")
            
except Exception as e:
    print(f"✗ Error with index management: {e}")

INDEX CREATION
✓ Index 'arxiv-papers-chunks' already exists

Current Statistics:
   Documents: 250
   Size: 4,896,706 bytes


## 3. Data Pipeline - Run Airflow DAG

The **arxiv_paper_ingestion** DAG automatically:
1. Fetches papers from arXiv API
2. Stores papers in PostgreSQL
3. **Indexes papers into OpenSearch**

### Instructions:

**Before proceeding, run the Airflow DAG:**

1. Open Airflow UI: http://localhost:8080
2. Login: username `admin`, password `admin`
3. Find **`arxiv_paper_ingestion`** DAG
4. Click the DAG name to open it
5. Click **"Trigger DAG"** button (▶️ play icon)
6. Wait ~10 minutes for completion
7. Check that all tasks turn green

Then run the cell below to verify:

In [6]:
# Verify Data Pipeline Results
print("VERIFYING DATA PIPELINE")
print("=" * 40)

stats = opensearch_client.get_index_stats()

if stats and 'error' not in stats:
    doc_count = stats.get('document_count', 0)
    
    if doc_count > 0:
        print(f"✓ Success! Found {doc_count} documents in OpenSearch")
        
        # Show sample papers
        sample = opensearch_client.search_papers("*", size=3)
        if sample.get('hits'):
            print(f"\nSample papers:")
            for i, paper in enumerate(sample['hits'], 1):
                title = paper.get('title', 'Unknown')[:60]
                print(f"  {i}. {title}...")
    else:
        print("⚠️  No documents in OpenSearch yet")
        print("\nPlease run the Airflow DAG first (see instructions above)")
else:
    print("✗ Could not retrieve index stats")

VERIFYING DATA PIPELINE
✓ Success! Found 250 documents in OpenSearch


## 4. Simple BM25 Search

Let's start with a simple search to demonstrate BM25 scoring:

In [7]:
# Simple BM25 Search
print("SIMPLE BM25 SEARCH")
print("=" * 40)

# Change this to any word from your papers
search_term = "learning"  # Try different terms!

print(f"Searching for: '{search_term}'\n")

results = opensearch_client.search_papers(
    query=search_term,
    size=5
)

if results.get('hits'):
    print(f"Found {results.get('total', 0)} total matches\n")
    
    for i, paper in enumerate(results['hits'], 1):
        print(f"{i}. {paper.get('title', 'Unknown')[:70]}...")
        print(f"   Score: {paper.get('score', 0):.2f}")
        print(f"   arXiv ID: {paper.get('arxiv_id', 'N/A')}\n")
else:
    print("No results found. Try searching for:")
    print("  • 'neural', 'model', 'algorithm'")
    print("  • Use '*' to see all papers")

SIMPLE BM25 SEARCH
Searching for: 'learning'

Found 80 total matches

1. MedROV: Towards Real-Time Open-Vocabulary Detection Across Diverse Med...
   Score: 5.71
   arXiv ID: 2511.20650v1

2. MapReduce LoRA: Advancing the Pareto Front in Multi-Preference Optimiz...
   Score: 5.79
   arXiv ID: 2511.20629v1

3. Fighting AI with AI: Leveraging Foundation Models for Assuring AI-Enab...
   Score: 5.63
   arXiv ID: 2511.20627v1

4. Fighting AI with AI: Leveraging Foundation Models for Assuring AI-Enab...
   Score: 2.82
   arXiv ID: 2511.20627v1

5. Fighting AI with AI: Leveraging Foundation Models for Assuring AI-Enab...
   Score: 2.67
   arXiv ID: 2511.20627v1



## 5. Advanced OpenSearch Queries

Now let's explore different query types using the OpenSearch Python client directly. This shows the power of BM25 without needing vectors!

### 5.1 Match Query

The `match` query is the standard query for full-text search on a single field:

In [8]:
# Match Query - Search in title field
print("MATCH QUERY - Single Field Search")
print("=" * 40)

query = {
    "query": {
        "match": {
            "title": "machine learning"
        }
    },
    "size": 3
}

response = opensearch_client.client.search(
    index=opensearch_client.index_name,
    body=query
)

print(f"Found {response['hits']['total']['value']} results\n")

for hit in response['hits']['hits']:
    print(f"Title: {hit['_source']['title'][:70]}...")

MATCH QUERY - Single Field Search
Found 25 results

Title: Bridging Philosophy and Machine Learning: A Structuralist Framework fo...
Title: Health system learning achieves generalist neuroimaging models...
Title: Stage-Specific Benchmarking of Deep Learning Models for Glioblastoma F...


### 5.2 Multi-Match Query

Search across multiple fields simultaneously:

In [9]:
# Multi-Match Query - Search across multiple fields
print("MULTI-MATCH QUERY - Search Multiple Fields")
print("=" * 40)

query = {
    "query": {
        "multi_match": {
            "query": "AI Agents",
            "fields": ["title^2", "abstract", "authors"],  # ^2 boosts title field
            "type": "best_fields"
        }
    },
    "size": 3
}

response = opensearch_client.client.search(
    index=opensearch_client.index_name,
    body=query
)

print(f"Found {response['hits']['total']['value']} results\n")

for hit in response['hits']['hits']:
    print(f"Title: {hit['_source']['title'][:70]}...")
    print(f"Score: {hit['_score']:.2f}")
    print(f"Authors: {', '.join(hit['_source']['authors'][:2])}...\n")

MULTI-MATCH QUERY - Search Multiple Fields
Found 135 results

Title: BrowseSafe: Understanding and Preventing Prompt Injection Within AI Br...
Score: 5.11
Authors: K, a...

Title: BrowseSafe: Understanding and Preventing Prompt Injection Within AI Br...
Score: 5.11
Authors: K, a...

Title: BrowseSafe: Understanding and Preventing Prompt Injection Within AI Br...
Score: 5.11
Authors: K, a...



### 5.3 Boosting Query

Boost certain results while demoting others:

In [10]:
# Boosting Query - Promote and demote results
print("BOOSTING QUERY - Promote/Demote Results")
print("=" * 40)

query = {
    "query": {
        "boosting": {
            "positive": {
                "match": {
                    "abstract": "deep learning"
                }
            },
            "negative": {
                "match": {
                    "abstract": "multimodal"
                }
            },
            "negative_boost": 0.1  # Reduce score of negative matches
        }
    },
    "size": 3
}

response = opensearch_client.client.search(
    index=opensearch_client.index_name,
    body=query
)

print(f"Query: Boost 'deep learning', demote 'survey' papers\n")
print(f"Found {response['hits']['total']['value']} results\n")

for hit in response['hits']['hits']:
    title = hit['_source']['title'][:70]
    abstract_snippet = hit['_source']['abstract'][:100]
    print(f"Title: {title}...")
    print(f"Score: {hit['_score']:.2f}")
    print(f"Abstract: {abstract_snippet}...\n")

BOOSTING QUERY - Promote/Demote Results
Query: Boost 'deep learning', demote 'survey' papers

Found 55 results

Title: Bridging Philosophy and Machine Learning: A Structuralist Framework fo...
Score: 6.13
Abstract: Machine learning models increasingly function as representational systems, yet the philosoph- ical a...

Title: Health system learning achieves generalist neuroimaging models...
Score: 5.39
Abstract: Frontier artificial intelligence (AI) models, such as OpenAI's GPT-5 and Meta's DINOv3, have advance...

Title: Universality in Collective Intelligence on the Rubik's Cube...
Score: 4.73
Abstract: Progress in understanding expert performance is limited by the scarcity of quantitative data on long...



### 5.4 Filter Query

Filter results by specific criteria (doesn't affect scoring):

In [11]:
# Filter Query - Filter by categories
print("FILTER QUERY - Category Filtering")
print("=" * 40)

query = {
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "abstract": "neural"
                    }
                }
            ],
            "filter": [
                {
                    "terms": {
                        "categories": ["cs.AI"]
                    }
                }
            ]
        }
    },
    "size": 3
}

response = opensearch_client.client.search(
    index=opensearch_client.index_name,
    body=query
)

print(f"Found {response['hits']['total']['value']} results\n")

for hit in response['hits']['hits']:
    title = hit['_source']['title'][:70]
    categories = ', '.join(hit['_source']['categories'])
    print(f"Title: {title}...")
    print(f"Categories: {categories}")
    print(f"Score: {hit['_score']:.2f}\n")

FILTER QUERY - Category Filtering
Found 33 results

Title: Evaluating the Performance of Deep Learning Models in Whole-body Dynam...
Categories: cs.CV, cs.AI
Score: 2.57

Title: Evaluating the Performance of Deep Learning Models in Whole-body Dynam...
Categories: cs.CV, cs.AI
Score: 2.57

Title: Evaluating the Performance of Deep Learning Models in Whole-body Dynam...
Categories: cs.CV, cs.AI
Score: 2.57



### 5.5 Sorting Query

Sort results by different criteria:

In [12]:
# Sorting Query - Sort by publication date
print("SORTING QUERY - Latest Papers First")
print("=" * 40)

query = {
    "query": {
        "match_all": {}  # Get all papers
    },
    "sort": [
        {
            "published_date": {
                "order": "desc"  # Latest first
            }
        }
    ],
    "size": 5
}

response = opensearch_client.client.search(
    index=opensearch_client.index_name,
    body=query
)

print(f"Query: All papers sorted by publication date (newest first)\n")

for hit in response['hits']['hits']:
    title = hit['_source']['title'][:70]
    pub_date = hit['_source']['published_date'][:10]
    print(f"Date: {pub_date} | {title}...")

SORTING QUERY - Latest Papers First
Query: All papers sorted by publication date (newest first)

Date: 2025-11-25 | MedROV: Towards Real-Time Open-Vocabulary Detection Across Diverse Med...
Date: 2025-11-25 | MotionV2V: Editing Motion in a Video...
Date: 2025-11-25 | Latent Collaboration in Multi-Agent Systems...
Date: 2025-11-25 | MapReduce LoRA: Advancing the Pareto Front in Multi-Preference Optimiz...
Date: 2025-11-25 | Fighting AI with AI: Leveraging Foundation Models for Assuring AI-Enab...


### 5.6 Combined Query

Combine multiple query types for complex searches:

In [13]:
# Combined Query - Complex search with multiple criteria
print("COMBINED QUERY - Complex Search")
print("=" * 40)

query = {
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": "transformer",
                        "fields": ["title^3", "abstract"],
                        "type": "best_fields"
                    }
                }
            ],
            "filter": [
                {
                    "range": {
                        "published_date": {
                            "gte": "2024-01-01"
                        }
                    }
                }
            ],
            "should": [
                {
                    "match": {
                        "categories": "cs.AI"
                    }
                }
            ]
        }
    },
    "sort": [
        "_score",
        {"published_date": {"order": "desc"}}
    ],
    "size": 3
}

response = opensearch_client.client.search(
    index=opensearch_client.index_name,
    body=query
)

print(f"Complex Query:")
print(f"  • Must contain 'transformer' (title boosted 3x)")
print(f"  • Filter: published after 2024-01-01")
print(f"  • Prefer: cs.AI category")
print(f"  • Sort: by relevance, then date\n")

print(f"Found {response['hits']['total']['value']} results\n")

for hit in response['hits']['hits']:
    title = hit['_source']['title'][:70]
    pub_date = hit['_source']['published_date'][:10]
    score = hit['_score']
    categories = ', '.join(hit['_source']['categories'][:2])
    
    print(f"Title: {title}...")
    print(f"  Date: {pub_date} | Score: {score:.2f}")
    print(f"  Categories: {categories}\n")

COMBINED QUERY - Complex Search
Complex Query:
  • Must contain 'transformer' (title boosted 3x)
  • Filter: published after 2024-01-01
  • Prefer: cs.AI category
  • Sort: by relevance, then date

Found 36 results

Title: Stage-Specific Benchmarking of Deep Learning Models for Glioblastoma F...
  Date: 2025-11-23 | Score: 2.54
  Categories: cs.CV, cs.AI

Title: Evaluating the Performance of Deep Learning Models in Whole-body Dynam...
  Date: 2025-11-25 | Score: 2.47
  Categories: cs.CV, cs.AI

Title: Evaluating the Performance of Deep Learning Models in Whole-body Dynam...
  Date: 2025-11-25 | Score: 2.47
  Categories: cs.CV, cs.AI



## Summary

### What We Demonstrated

**BM25 Search is Powerful!** Without any vector embeddings, we can:

1. **Simple Search**: Basic keyword search with relevance scoring
2. **Match Queries**: Search specific fields
3. **Multi-Match**: Search across multiple fields with boosting
4. **Boosting**: Promote or demote certain results
5. **Filtering**: Apply filters without affecting scores
6. **Sorting**: Order results by date, score, or other fields
7. **Complex Queries**: Combine all techniques for sophisticated searches

### Key Takeaways

- **BM25 works great** for many search use cases
- **No vectors needed** for effective full-text search
- **Simple and fast** compared to embedding-based approaches
- **Filters and sorting** make searches precise and relevant
- **Field boosting** helps prioritize important content

### When to Use BM25 vs Vectors

**Use BM25 when:**
- Searching for specific keywords or phrases
- Need fast, simple implementation
- Have good text fields with clear terminology
- Want explainable search results

**Consider vectors when:**
- Need semantic similarity (concepts, not keywords)
- Dealing with synonyms and paraphrasing
- Cross-language search requirements
- Very short queries or documents

Remember: **You can also combine both** (hybrid search) for best results!
We will see this in the next week :)

In [ ]:
# In your Week 3 notebook or new cell:
import requests

# Check OpenSearch
response = requests.get("http://localhost:9200/arxiv-papers-chunks/_count")
chunk_count = response.json()['count']
print(f"✓ Indexed chunks: {chunk_count}")

# Check PostgreSQL
import psycopg2
conn = psycopg2.connect(
    host='localhost',
    port=5432,
    database='rag_db',
    user='rag_user',
    password='rag_password'
)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM papers")
paper_count = cursor.fetchone()[0]
print(f"✓ Papers in database: {paper_count}")
conn.close()

In [ ]:
# In your notebook
import psycopg2

conn = psycopg2.connect(
    host='localhost',
    port=5432,
    database='rag_db',
    user='rag_user',
    password='rag_password'
)

cursor = conn.cursor()

# Check if papers have text content
cursor.execute("""
    SELECT id, arxiv_id, title, 
           LENGTH(raw_text) as text_length,
           LENGTH(abstract) as abstract_length
    FROM papers 
    LIMIT 5
""")

for row in cursor.fetchall():
    print(f"Paper {row[1]}: text={row[3]} chars, abstract={row[4]} chars")

conn.close()

In [ ]:
# In your notebook - this will show task results
import psycopg2
import json

conn = psycopg2.connect(
    host='localhost', port=5432,
    database='rag_db', user='rag_user', password='rag_password'
)

cursor = conn.cursor()

# Get recent task results from XCom
cursor.execute("""
    SELECT task_id, key, value::text, timestamp
    FROM xcom 
    WHERE dag_id = 'arxiv_paper_ingestion'
    ORDER BY timestamp DESC
    LIMIT 30
""")

print("Recent Task Results:")
print("=" * 80)
for row in cursor.fetchall():
    print(f"\nTask: {row[0]}")
    print(f"Key: {row[1]}")
    print(f"Time: {row[3]}")
    print(f"Value: {row[2][:500]}...")  # First 500 chars

conn.close()

In [ ]:
# Run this in your Week 3 notebook
import psycopg2
from opensearchpy import OpenSearch
from datetime import datetime

conn = psycopg2.connect(host='localhost', port=5432, database='rag_db', user='rag_user', password='rag_password')
cursor = conn.cursor()
cursor.execute("SELECT id, arxiv_id, title, authors, abstract, categories, published_date FROM papers WHERE abstract IS NOT NULL")
papers = cursor.fetchall()
conn.close()

client = OpenSearch(hosts=[{'host': 'localhost', 'port': 9200}], use_ssl=False)

for paper in papers:
    doc = {
        'chunk_id': f"{paper[1]}_abstract", 'arxiv_id': paper[1], 'paper_id': str(paper[0]),
        'chunk_index': 0, 'chunk_text': paper[4], 'chunk_word_count': len(paper[4].split()),
        'title': paper[2], 'authors': paper[3], 'abstract': paper[4],
        'categories': paper[5], 'published_date': paper[6],
        'section_title': 'Abstract', 'created_at': datetime.now().isoformat()
    }
    client.index(index='arxiv-papers-chunks', id=doc['chunk_id'], body=doc, refresh=True)

print(f"✅ Indexed {client.count(index='arxiv-papers-chunks')['count']} abstracts!")

In [ ]:
from opensearchpy import OpenSearch

client = OpenSearch(hosts=[{'host': 'localhost', 'port': 9200}], use_ssl=False)

# Test BM25 search
query = {
    "query": {
        "match": {
            "chunk_text": "transformer neural network"
        }
    },
    "size": 5
}

results = client.search(index='arxiv-papers-chunks', body=query)

print(f"Found {results['hits']['total']['value']} results")
print("\nTop results:")
for hit in results['hits']['hits']:
    print(f"\nScore: {hit['_score']:.2f}")
    print(f"Title: {hit['_source']['title']}")
    print(f"Text: {hit['_source']['chunk_text'][:200]}...")

In [ ]:
# In your notebook - check if papers have full text now
import psycopg2

conn = psycopg2.connect(
    host='localhost',
    port=5432,
    database='rag_db',
    user='rag_user',
    password='rag_password'
)

cursor = conn.cursor()

# Check if papers have text content
cursor.execute("""
    SELECT arxiv_id, 
           LENGTH(raw_text) as text_length,
           LENGTH(abstract) as abstract_length
    FROM papers 
    WHERE raw_text IS NOT NULL
    ORDER BY id DESC
    LIMIT 10
""")

print("Papers with full text:")
for row in cursor.fetchall():
    print(f"  {row[0]}: text={row[1]} chars, abstract={row[2]} chars")

cursor.execute("SELECT COUNT(*) FROM papers WHERE raw_text IS NOT NULL")
count_with_text = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM papers")
total_papers = cursor.fetchone()[0]

print(f"\n✓ {count_with_text}/{total_papers} papers have full text")

conn.close()

# Check OpenSearch chunks
import requests
response = requests.get("http://localhost:9200/arxiv-papers-chunks/_count")
print(f"✓ OpenSearch has {response.json()['count']} chunks indexed")

In [ ]:
# Run this in your notebook
import psycopg2

conn = psycopg2.connect(
    host='localhost', port=5432,
    database='rag_db', user='rag_user', password='rag_password'
)

cursor = conn.cursor()

cursor.execute("""
    SELECT arxiv_id, LENGTH(raw_text) as text_length
    FROM papers 
    WHERE raw_text IS NOT NULL
    ORDER BY id DESC
    LIMIT 10
""")

print("Papers with full text:")
for row in cursor.fetchall():
    print(f"  {row[0]}: {row[1]:,} characters")

cursor.execute("SELECT COUNT(*) FROM papers WHERE raw_text IS NOT NULL")
count = cursor.fetchone()[0]
print(f"\n✓ {count} papers have full text!")

conn.close()

# Check OpenSearch after indexing task completes
import requests
response = requests.get("http://localhost:9200/arxiv-papers-chunks/_count")
print(f"✓ OpenSearch will have chunks after indexing task completes")

In [ ]:
# After DAG completes, check chunks
import requests

response = requests.get("http://localhost:9200/arxiv-papers-chunks/_count")
chunk_count = response.json()['count']
print(f"✓ OpenSearch has {chunk_count} chunks indexed")

# Search for something in the new papers
query = {
    "query": {"match": {"chunk_text": "neural network"}},
    "size": 3
}

from opensearchpy import OpenSearch
client = OpenSearch(hosts=[{'host': 'localhost', 'port': 9200}], use_ssl=False)
results = client.search(index='arxiv-papers-chunks', body=query)

print(f"\nFound {results['hits']['total']['value']} results")
for hit in results['hits']['hits']:
    print(f"\nScore: {hit['_score']:.2f}")
    print(f"Title: {hit['_source']['title']}")
    print(f"Text: {hit['_source']['chunk_text'][:150]}...")

In [ ]:
# Check if the 8 parsed papers are being seen by the indexing task
import psycopg2

conn = psycopg2.connect(
    host='localhost', port=5432,
    database='rag_db', user='rag_user', password='rag_password'
)

cursor = conn.cursor()

# Check what the indexing task queries
cursor.execute("""
    SELECT COUNT(*) 
    FROM papers 
    WHERE raw_text IS NOT NULL 
    AND created_at >= NOW() - INTERVAL '1 day'
""")

recent_with_text = cursor.fetchone()[0]
print(f"Papers with text from last 24h: {recent_with_text}")

# Check the 8 papers' created_at dates
cursor.execute("""
    SELECT arxiv_id, created_at, LENGTH(raw_text) 
    FROM papers 
    WHERE raw_text IS NOT NULL 
    ORDER BY created_at DESC 
    LIMIT 8
""")

print("\nRecent papers with text:")
for row in cursor.fetchall():
    print(f"  {row[0]}: created {row[1]}, {row[2]:,} chars")

conn.close()

In [ ]:
# Check the latest DAG run results
import psycopg2

conn = psycopg2.connect(
    host='localhost', port=5432,
    database='rag_db', user='rag_user', password='rag_password'
)

cursor = conn.cursor()

# Get latest indexing stats from XCom
cursor.execute("""
    SELECT value::text
    FROM xcom 
    WHERE dag_id = 'arxiv_paper_ingestion'
      AND task_id = 'index_papers_hybrid'
      AND key = 'hybrid_index_stats'
    ORDER BY timestamp DESC
    LIMIT 1
""")

result = cursor.fetchone()
if result:
    import binascii, json
    hex_data = result[0][2:]
    decoded = binascii.unhexlify(hex_data).decode('utf-8')
    data = json.loads(decoded)
    print("Latest indexing stats:")
    print(json.dumps(data, indent=2))

conn.close()

In [ ]:
# Check for chunks from the newly parsed papers
from opensearchpy import OpenSearch

client = OpenSearch(hosts=[{'host': 'localhost', 'port': 9200}], use_ssl=False)

# Search for papers from today's batch (2511.20xxx)
query = {
    "query": {
        "prefix": {
            "arxiv_id": "2511.20"
        }
    },
    "size": 10
}

results = client.search(index='arxiv-papers-chunks', body=query)
print(f"Chunks from newly parsed papers (2511.20xxx): {results['hits']['total']['value']}")

for hit in results['hits']['hits']:
    print(f"  {hit['_source']['arxiv_id']}: {hit['_source'].get('chunk_index', 'N/A')}")

In [14]:
import psycopg2, binascii, json

conn = psycopg2.connect(host='localhost', port=5432, database='rag_db', user='rag_user', password='rag_password')
cursor = conn.cursor()

cursor.execute("""
    SELECT value::text FROM xcom 
    WHERE dag_id = 'arxiv_paper_ingestion'
      AND task_id = 'index_papers_hybrid'
      AND key = 'hybrid_index_stats'
    ORDER BY timestamp DESC LIMIT 1
""")

result = cursor.fetchone()
if result:
    hex_data = result[0][2:]
    decoded = binascii.unhexlify(hex_data).decode('utf-8')
    data = json.loads(decoded)
    print("\nIndexing stats:")
    print(json.dumps(data, indent=2))
conn.close()


Indexing stats:
{
  "papers_processed": 10,
  "total_chunks_created": 230,
  "total_chunks_indexed": 230,
  "total_embeddings_generated": 230,
  "total_errors": 0
}
